In [ ]:
!pip install mne
!pip install pyedflib

In [2]:
import pyedflib

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [5]:
#edit with your edf and rml file paths
edf_file_path = "/content/drive/MyDrive/CSE-578_DV/Sleep Apnea Project/00001000-100507[002].edf"
annot_file_path = "/content/drive/MyDrive/CSE-578_DV/Sleep Apnea Project/00001000-100507.rml"

In [ ]:
f = pyedflib.EdfReader(edf_file_path)
# Get the number of signals (channels)
num_signals = f.signals_in_file

# Read signals
signals = [f.readSignal(i) for i in range(num_signals)]

# Get signal labels
signal_labels = f.getSignalLabels()

print("No. of Signals: ",num_signals)
print("Signal Labels: ",signal_labels)
f.close()

In [ ]:
import os


annot_file_path = "/content/drive/MyDrive/CSE-578_DV/Sleep Apnea Project/00001000-100507.rml" #replace with rml file path
with open(annot_file_path, 'r') as fileID:
    strlines = fileID.readlines()

strlinesv = [line.split() for line in strlines]

family = []
family_type = []
start = []
duration = []
segment_files = 0
# starting_date = 0
# total_duration = 0

# print(strlinesv)

for temp in strlinesv:
    if len(temp) > 1:
        if temp[0] == '<Event':
            familyv = temp[1].split('=')
            family.append(familyv[0].strip('"'))

            family_typev = temp[2].split('=')
            family_type.append(family_typev[1].strip('"'))

            startv = temp[3].split('=')
            start.append(float(startv[1].strip('"')))

            durationv = temp[4].split('=')
            duration.append(float(durationv[1].replace(">","").strip('"')))
        else:
            if temp[0] == '<Session>':
                starting_time_line = strlinesv[strlinesv.index(temp) + 1]
                temp_start_time = starting_time_line[0].split('<T>')
                starting_date = temp_start_time[1]

                total_duration_time = strlinesv[strlinesv.index(temp) + 2]
                temp_total_duration = total_duration_time[0].split('</Duration>')
                total_duration = float(temp_total_duration[0].split('<Duration>')[0])
            else:
                if temp[0] == '<Segment>':
                    segment_files += 1

In [ ]:
signals_dict = {}
events_list = []
starts_list = []
durations_list = []
spo2_list = []
pr_list = []
for event in [(family_type[idx],start[idx],duration[idx]) for idx in range(1111) if "pnea" in family_type[idx]]:
  print(event)
  edf_file_path = "/content/drive/MyDrive/CSE-578_DV/Sleep Apnea Project/00001000-100507[00"+str(int(event[1]//3600)+1)+"].edf" #replace with edf file paths, better place all edf files and execute the code
  print("EDF File To be Examined: ",edf_file_path)
  f = pyedflib.EdfReader(edf_file_path)
  # Get the number of signals (channels)
  num_signals = f.signals_in_file

  # Read signals
  signals = [f.readSignal(i) for i in range(num_signals)]

  # Get signal labels
  signal_labels = f.getSignalLabels()

  # print("No. of Signals: ",num_signals)
  # print("Signal Labels: ",signal_labels)
  f.close()
  for index in range(len(signal_labels)):
    signals_dict[signal_labels[index]] = index
  spo2_index = signals_dict['SpO2']
  pulse_index = signals_dict['PulseRate']

  start = int(event[1]) - (3600*(int(event[1]//3600)))
  end = start+int(event[2])
  # print(start,end,sep="-->")
  events_list.append(event[0])
  starts_list.append(event[1])
  durations_list.append(event[2])
  # print("SPO2 values during "+str(event[0])+" are: ",signals[spo2_index][start:start+int(event[2])])
  # print("Pulse Rate values during "+str(event[0])+" are: ",signals[pulse_index][start:start+int(event[2])])
  spo2_list.append(signals[spo2_index][start:start+int(event[2])])
  pr_list.append(signals[pulse_index][start:start+int(event[2])])




In [ ]:
signals_info = pd.DataFrame()
signals_info["Event"] = events_list
signals_info["Start"] = starts_list
signals_info["Duration"] = durations_list
signals_info["Spo2"] = spo2_list
signals_info["Pulse Rate"] = pr_list
signals_info["Patient_ID"] = [995]*len(pr_list)

signals_info.to_csv("signals_info_999.csv",index=False)

In [ ]:
with open(annot_file_path, 'r') as fileID:
    strlines = fileID.readlines()

strlinesv = [line.split() for line in strlines]

family = []
family_type = []
start = []
duration = []
segment_files = 0
# starting_date = 0
# total_duration = 0

# print(strlinesv)
stages_list = []
durations_list = []
for temp in strlinesv:
  if len(temp) > 1:
    if temp[0] == '<Stage':
      stages_list.append(temp[1].split("=")[-1].replace('"',''))
      durations_list.append(int(temp[2].split("=")[-1].replace('"','')))

In [ ]:
import pandas as pd
stages_df = pd.DataFrame()
stages_df["Stage"] = stages_list[:-3]
stages_df["Start"] = durations_list[:-3]
stages_df["Patient_ID"] = [995]*len(stages_list[:-3])
stages_df.to_csv("stage_data_999.csv",index=False)